# language tools

Converting between languages, parsing languages computer languages, demonstrating tools like syntax hilighters, and outputting the diffed code


TODO: use syntax tree to convert from and to any language


use a common library for language generation 

https://bitbucket.org/birkenfeld/pygments-main/src/default/pygments/lexers/haskell.py

https://github.com/PrismJS/prism/blob/master/components/prism-markdown.js

http://qbnz.com/highlighter/

https://stackoverflow.com/questions/171924/how-do-i-syntax-check-a-bash-script-without-running-it

bash -n scriptname

Improve on the babel-preset-php pattern of replacing code with a tree, just use templates and the tree creator on regular code blocks

https://gitlab.com/kornelski/babel-preset-php/blob/master/lib/trans.js

```
function translateAssignBlock(node, scope) {
    if (isVariableCreatingAssignment(node, scope)) {
        return makeVariableDeclaration(node.left.name, "var", translateExpr(node.right, scope), node.loc, scope);
    }
    return wrapInStatement(translateAssign(node, scope));
}
```

should be something like:
```
var ASSIGMENT_EXPRESSION = (leftName, right) => {
    var leftName = right;
}

function translateAssignBlock(node, scope) {
    if(//Assignment[/Identifier].includes(node)) {
        return getAST(ASSIGMENT_EXPRESSION, node)
    }
    return getAST(STATEMENT, node)
}
```

that way when they are written in multiple languages we can transpile any language, maybe

use this library as a reference for lots of languages https://github.com/seaneshbaugh/rosetta-euler

automatically translate https://projecteuler.net/archives to any language


TODO: use babel and a transpiling kernel from babel-preset-php, 

TODO: make a generic translator that does manipulations on queries based on templates






In [ ]:
// sample definition for MatLab



In [ ]:
// definition for nodejs



## TODO: babel transpiler plugin



## syntax tree tools?




### justification

Just so it can be parsed with XPath instead. Get them all working the same!




Crap modules I won't need any more:
- css
- ASTQ
- Acorn
- esprima
- escodegen
- esquery
- jsonpath
- jsonquery
- jspath
- json:select
- jsel

TODO: get rid of jsonpath-plus, jsonstream, jspath, 



```
/* expected output `
[ Script {
    type: 'Program',
    body: [ [FunctionDeclaration], [ExpressionStatement] ],
    sourceType: 'script',
    range: [ 1, 45 ],
    comments: [],
    tokens:

<program type="program">
    <function>
    </function>
    <expression>
    </expression>
</program>

*/
```



### convert a syntax tree to html?

Reasoning. Every atom is made up of smaller component parts. This conversion creates a tree that breaks down to &lt;elements&gt; and attr="ibutes", just like neutrons and protons.


In [2]:

function statementToElement(statement) {
    var {type} = statement;
    var result = ``, attrs = ``;
    var isList = true;
    for(var i in statement) {
        if(parseInt(i) + '' === i + '') {
            result += statementToElement(statement[i])
            continue;
        }
        isList = false;
        var jsType = typeof statement[i];
        if(jsType === 'object' && statement[i]) {
            jsType = Object.getPrototypeOf(statement[i]).constructor.name;
        }
        // if the property is an Object, print out as a child
        // TODO: replace this type property when converting other trees
        //   should be like getEl() statementToElement(() => el.type)
        if(typeof statement[i].type !== 'undefined'
           || (typeof statement[i][0] !== 'undefined'
        // or if the property is an Array, print them out as child elements
           && typeof statement[i][0].type !== 'undefined')) {
            // print out the other keys just for fun matching
            attrs +=  `
${i}="" ${i}-type="${jsType}"`;
            result += `<${i}-attr>${statementToElement(statement[i])}</${i}-attr>`
        } else {
            // if the property is not an object, i.e. native types
            attrs +=  `
${i}="${statement[i]}" ${i}-type="${jsType}"`
        }
    }
    
    return isList ? result : `
<${type}${attrs}>${result}</${type}>`
}

module.exports = statementToElement;


[Function: statementToElement]

### convert an html tree to objects?



In [9]:
var {JSDOM} = require('jsdom');

// TODO: create a pattern same as notebook markdown in core, accumulate
function accumulateComments(nodes) {
    var commentBuffer = []
    return nodes.reduce((acc, n) => {
        if(typeof n !== 'object') {
            commentBuffer.push[n];
        } else {
            n.comments = commentBuffer;
            commentBuffer = [];
            acc.push(n);
        }
        return acc;
    }, []);
}

// expects a string or a tree from JSDOM
function elementToStatement(body) {
    if(typeof body === 'string') {
        var dom = new JSDOM(body);
        body = dom.window.document.body;
    }
    if(body.nodeName === '#text') {
        return body.nodeValue;
    }
    var attrs = body.getAttributeNames()
        .filter(a => a.substr(-5) !== '-type');
    if(attrs.length === 0) {
        var elsAndText = Array.from(body.childNodes)
            .map(c => elementToStatement(c));
        return accumulateComments(elsAndText);
    }
    var result = attrs.reduce((obj, p) => {
        var childObject = Array.from(body.childNodes)
           .filter(n => n.nodeName.toLowerCase() === p.toLowerCase() + '-attr')[0];
        var type = body.getAttribute(p + '-type');
        if(childObject) {
            // converted from an array of childNodes above
            var children = elementToStatement(childObject);
            obj[p] = type === 'Array' ? children : children[0];
        } else {
            var attr = body.getAttribute(p);
            obj[p] = attr;
            if(type === 'boolean')  obj[p] = attr === 'true';
            if(type === 'number')  obj[p] = parseInt(attr);
        }
        return obj;
    }, {})
    return result;
}

module.exports = elementToStatement;



[Function: elementToStatement]

#### test tree to html?

In [11]:
var importer = require('../Core');
var assert = require('assert');
// TODO: make this dependency obsolete
var esquery = require('esquery');
var esprima = require('esprima'); 
var escodegen = require('escodegen');
var ASTQ  = require("astq");
var astq = new ASTQ();

var gen_options = {
    comment: true,
    tokens: true,
    whitespace: true,
    format: {indent: {style: '    '}, quotes: 'single'}
}

function setupASTArray(code) {
    var ctx = esprima.parse(code, {range: true, tokens: true,
                        comment: true, whitespace: true})
    
    escodegen.attachComments(ctx, ctx.comments, ctx.tokens, ctx.whitespace);
    return ctx;
}

function testASTArray(ctx) {
    var output1 = escodegen.generate(ctx, gen_options)
    console.log(output1);
    var page = statementToElement(ctx);
    var translated_back = elementToStatement(page)[0];
    var output2 = escodegen.generate(translated_back, gen_options);
    assert(output1 === output2, 'different code trees');    
    return page;
}

module.exports = testASTArray;

function htmlEntities(str) {
    return String(str)
        .replace(/&/g, '&amp;')
        .replace(/</g, '&lt;')
        .replace(/>/g, '&gt;')
        .replace(/"/g, '&quot;');
}

if(typeof $$ !== 'undefined') {
    var code = `
function name(params) {
    
}
console.log()
`
    var ctx = setupASTArray(code);
    var html = testASTArray(ctx)
    console.log(astq.query(ctx, '*'));
    console.log(astq.query(ctx, '*')[0].tokens);
    console.log(esquery(ctx, '*').length);
    console.log(esquery(ctx, 'Program').length);
    
    $$.html('<pre>' + htmlEntities(html) + '</pre>')
}


function name(params) {
}
console.log();


Error: Unknown node type: undefined

#### TODO test tree to html with astq

TODO: convert between astq and other tree libraries

replacing that functionality I noticed in babel somewhere



In [106]:
// reserved words

var keywords = ['type', 'id', 'params',
                'body', 'generator', 'async',
                'range', 'expression', 'callee',
                'arguments'];



expected output


```
FunctionDeclaration {
    type: 'FunctionDeclaration',
    id:
     Identifier { type: 'Identifier', name: 'name', range: [Array] },
    params: [ [Identifier] ],
    body:
     BlockStatement { type: 'BlockStatement', body: [], range: [Array] },
    generator: false,
    expression: false,
    async: false,
    range: [ 1, 31 ] },
  ExpressionStatement {
    type: 'ExpressionStatement',
    expression:
     CallExpression {
       type: 'CallExpression',
       callee: [StaticMemberExpression],
       arguments: [],
       range: [Array] },
    range: [ 32, 45 ] } ]
```


### generate a synax tree from documentation


#### test language translator

TODO: Creating an interface like this becomes a simple http request to download and create a language compatible pattern, do in C# and in Node




In [ ]:

var wireKernelInterface = {
    ...metaKernelInterface,
    // implement all requests, these are required by the meta kernel
    execute_request, inspect_request, complete_request,
    history_request, is_complete_request, shutdown_request,
    // requests maybe not required right now?
    kernel_info_request, interrupt_request, input_request,
    connect_request, comm_info_request, kernel_info_request,

    // implement all replys
    execute_reply, inspect_reply, complete_reply,
    history_reply, kernel_info_reply, is_complete_reply,
    connect_reply, comm_info_reply, kernel_info_reply,
    shutdown_reply, interrupt_reply, input_reply,

    // a few extra protocol methods
    display_data, update_display_data, execute_input,
    execute_result, error, status, clear_output,
    comm_msg, comm_close
}

module.exports = wireKernelInterface;
    